#Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset

#Data Downloading and Feature-Target Separation

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)


train_features = train_dataset.data.unsqueeze(1).float() / 255.0
train_targets = train_dataset.targets

test_features = test_dataset.data.unsqueeze(1).float() / 255.0
test_targets = test_dataset.targets

train_features = (train_features - 0.5) / 0.5
test_features = (test_features - 0.5) / 0.5

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:02<00:00, 4.57MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 135kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.28MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.84MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



#PGD Attack

Projected Gradient Descent function generates adversarial examples by iteratively perturbing the input images within a specified epsilon (eps) and updates are made according to the gradient of the loss concerning the input image.

In [ ]:
def pgd_attack(model, images, labels, eps=0.3, alpha=0.01, iters=40):
    model.eval()
    images = images.to(device)
    labels = labels.to(device)
    ori_images = images.data

    for _ in range(iters):
        images.requires_grad = True
        outputs = model(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        model.zero_grad()
        loss.backward()
        adv_images = images + alpha * images.grad.sign()
        eta = torch.clamp(adv_images - ori_images, min=-eps, max=eps)
        images = torch.clamp(ori_images + eta, min=0, max=1).detach_()

    return images

#Deep CNN model

DeepCNN has 4 convolutional layers with ReLU activations,2 max-pooling layers for down-sampling and Fully connected layers with Dropout to prevent overfitting.

In [ ]:
class DeepCNN(nn.Module):
    def __init__(self):
        super(DeepCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)  # Output: 28x28x64
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)  # Output: 28x28x128
        self.relu2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 14x14x128

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)  # Output: 14x14x256
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)  # Output: 14x14x512
        self.relu4 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 7x7x512

        self.fc1 = nn.Linear(7 * 7 * 512, 1024)
        self.relu5 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 512)
        self.relu6 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.pool1(x)
        x = self.relu3(self.conv3(x))
        x = self.relu4(self.conv4(x))
        x = self.pool2(x)
        x = x.view(-1, 7 * 7 * 512)  # Flatten
        x = self.dropout1(self.relu5(self.fc1(x)))
        x = self.dropout2(self.relu6(self.fc2(x)))
        x = self.fc3(x)
        return x


#Initialize the model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepCNN().to(device)


#Clubbing perturbed and clean features

In [ ]:
def generate_new_dataset(model, features, targets, eps=0.3, alpha=0.01, iters=40):

    original_features = features.clone()
    targets = targets.clone()


    perturbed_features = []
    batch_size = 128
    model.eval()

    for i in range(0, len(features), batch_size):
        batch_features = features[i:i+batch_size].to(device)
        batch_targets = targets[i:i+batch_size].to(device)
        perturbed_batch = pgd_attack(model, batch_features, batch_targets, eps, alpha, iters)
        perturbed_features.append(perturbed_batch.cpu())

    perturbed_features = torch.cat(perturbed_features, dim=0)


    combined_features = torch.cat([original_features, perturbed_features], dim=0)
    combined_targets = torch.cat([targets, targets], dim=0)

    return TensorDataset(combined_features, combined_targets)

In [ ]:
# Generate perturbed training and testing datasets
new_train_dataset = generate_new_dataset(model, train_features, train_targets, eps=0.3, alpha=0.01, iters=40)
new_test_dataset = generate_new_dataset(model, test_features, test_targets, eps=0.3, alpha=0.01, iters=40)

# Create DataLoader for both new training and testing datasets
new_train_loader = DataLoader(new_train_dataset, batch_size=64, shuffle=True)
new_test_loader = DataLoader(new_test_dataset, batch_size=64, shuffle=False)

#Training

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

def train_model(model, train_loader, epochs=5):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        correct = 0
        total = 0

        for data, target in train_loader:
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            _, predicted = outputs.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

        print(f"Epoch {epoch + 1}: Loss = {epoch_loss:.4f}, Accuracy = {100. * correct / total:.2f}%")

train_model(model, new_train_loader, epochs=5)

Epoch 1: Loss = 33.5041, Accuracy = 99.69%
Epoch 2: Loss = 25.8241, Accuracy = 99.72%
Epoch 3: Loss = 29.0396, Accuracy = 99.70%
Epoch 4: Loss = 23.9841, Accuracy = 99.73%
Epoch 5: Loss = 28.9961, Accuracy = 99.68%


#Evaluation

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = outputs.max(1)
            correct += predicted.eq(target).sum().item()
            total += target.size(0)

    print(f"Test Accuracy: {100. * correct / total:.2f}%")

test_loader = DataLoader(TensorDataset(test_features, test_targets), batch_size=64, shuffle=False)

evaluate_model(model, new_test_loader)

Test Accuracy: 99.41%
